In [ ]:
import pandas as pd
import dataclasses
from tqdm import tqdm
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, confusion_matrix
import graphviz
from sklearn.model_selection import GridSearchCV
import numpy as np
import requests
from geopy.geocoders import Nominatim
import geocoder
import dataclasses
from tqdm import tqdm
import os
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import matplotlib as mat
import matplotlib.cm as cm
import numpy as np
from datetime import datetime
from collections import Counter
%matplotlib inline

In [ ]:
class Row:
    _id: str
    case_number: str
    date: str
    block: str
    IUSR: str
    primary_type: str
    description: str
    location_description: str
    arrest: str
    domestic: str
    beat: str
    district: str
    ward: str
    community_area: str
    fbi_code: str
    x_coordinate: str
    y_coordinate: str
    year: str
    updated_on: str
    latitude: str
    longitude: str

    def location(self) -> (str, str):
        return self.longitude, self.latitude

    def to_dict(self):
        return {
       '_id': self._id,
       'case_number': self.case_number,
       'date': self.date,
       'block': self.block,
       'IUSR': self.IUSR,
       'primary_type': self.primary_type,
       'description': self.description,
       'location_description': self.location_description,
       'arrest': self.arrest,
       'domestic': self.domestic,
       'beat': self.beat,
       'district': self.district,
       'ward': self.ward,
       'community_area': self.community_area,
       'fbi_code': self.fbi_code,
       'x_coordinate': self.x_coordinate,
       'y_coordinate': self.y_coordinate,
       'year': self.year,
       'updated_on': self.updated_on,
       'latitude': self.latitude,
       'longitude': self.longitude,
       'location': self.location
        }

website = 'https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present-Dashboard/5cd6-ry5g'
path = '\Desktop'

options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36")
driver = webdriver.Chrome(options=options)
driver.get(website)
time.sleep(10)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(1)
cell_counter = 0
attrs = []
rows = []

the_table = driver.find_element(By.CLASS_NAME, 'table-body')
driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", the_table)
scroll_increment = 20000
for _ in range(5):
    scroll_script = f"arguments[0].scrollTop += {scroll_increment};"
    driver.execute_script(scroll_script, the_table)
    time.sleep(5)  # Wait for the new data to load

# Get the total number of elements      #div table-body
total_elements = len(driver.find_elements(By.CLASS_NAME, 'cell-content'))
time.sleep(60)
# Initialize tqdm progress bar
pbar = tqdm(total=total_elements, desc='Progress', unit='col')

for content in driver.find_elements(By.CLASS_NAME, 'cell-content'):
    try:
        content = content.text
        attrs.append(content)
        cell_counter += 1

        if cell_counter > 21:
            attrs.pop()
            rows.append(Row(*attrs))
            attrs = []
            cell_counter = 0

        # Update the progress bar
        pbar.update(1)
    except:
        continue

# Close the progress bar
pbar.close()

for row in rows:
    print(row)

df = pd.DataFrame([obj.to_dict() for obj in rows])
df.to_csv('testingData.csv')
driver.quit()

In [ ]:
#########################good data cleaning ##################################################################

the_path='D:/programs/CHROM EXECUTEBLES/files for project/testingData.csv'
try:
    the_file=pd.read_csv(the_path)
    print("File saved at:", os.path.abspath('testingData2.csv'))
    print("File saved successfully.")
except Exception as e:
    print("Error occurred while saving the file:", e)


#the_file.head()
upgraded_one=the_file.iloc[:,1:]

colm_names=upgraded_one.columns.tolist()
colm_numbers=len(colm_names)
print(colm_numbers)
print(colm_names)





un_wanted_colms=['IUCR','Block','Beat','District','Ward','Community Area','Year','Updated On','Latitude','Longitude','Police Districts','Boundaries - ZIP Codes','Wards','Census Tracts','Police Beats','Zip Codes','Historical Wards 2003-2015','Community Areas','X Coordinate','Y Coordinate']
upgraded_one=upgraded_one.drop(un_wanted_colms,axis=1)

for colm in upgraded_one.columns :
    dups = upgraded_one[colm].duplicated()
    duplicate_count = dups.sum()
    if duplicate_count > 0 :
        print(f"Found {duplicate_count} duplicates in column '{colm}'.")

dups=upgraded_one['Case Number'].duplicated()
if dups.any():
    pgraded_one = upgraded_one.drop_duplicates(subset=[colm[0]])
else:
    print("no dups in Case Number")


for colmm in upgraded_one.columns :
    number_of_missing_vals = sum(upgraded_one[colmm].isnull() | (upgraded_one[colmm] == ''))
    if number_of_missing_vals > 0 :
        print(f"Column '{colmm}' has {number_of_missing_vals} missing value(s).")

value1="UNknown"

replacement_values = {'Location Description' : value1, 'Location' : value1,}
upgraded_one = upgraded_one.fillna(replacement_values)


column_name = 'Primary Type'
unique_values = upgraded_one[column_name].value_counts()

# Calculate the percentage of each unique value
total_count = unique_values.sum()
percentages = unique_values / total_count * 100

# Filter out values below the threshold
threshold = 1
filtered_values = unique_values[percentages >= threshold]

plt.figure(figsize=(8, 8))
plt.pie(filtered_values.values, labels=filtered_values.index, autopct='%1.1f%%')
plt.title(f'Unique Value Distribution for Column: {column_name} (Threshold: {threshold}%)')
plt.axis('equal')
plt.show()



the_data = upgraded_one['Date'].str.split(' ')
upgraded_one['Time'] = the_data.str[1]
upgraded_one['Seconds'] = upgraded_one['Time'].str.split(':').str[-1]
upgraded_one['Hour'] = upgraded_one['Time'].str.split(':').str[:1].str.join(':')

grouped_data = upgraded_one.groupby(['Primary Type', 'Hour']).size().unstack()

# Get the top six types
top_six_types = upgraded_one['Primary Type'].value_counts().head(6).index

# Plot the line chart for each primary type
for primary_type in top_six_types:
    data = grouped_data.loc[primary_type]
    plt.plot(data.index, data.values, marker='o', label=primary_type)

# Set the x-axis labels to the actual hour values
hours = [datetime.strptime(str(hour), "%H").strftime("%I %p") for hour in range(24)]
plt.xticks(range(24), hours)

# Set the x-axis limits to include all hours
plt.xlim(0, 23)

plt.xlabel('Hour')
plt.ylabel('Frequency')
plt.title('Correlation between Primary Type and Hour')
plt.legend()
plt.grid(True)
plt.show()


path_to_save='D:/programs/CHROM EXECUTEBLES/files for project/upgraded_file.csv'
upgraded_one.to_csv(path_to_save, index=False)

print(upgraded_one.columns)
print(len(upgraded_one.columns))

In [ ]:
##########some more cleaning #####

path_to_file = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big_saparated.csv'

try:
     the_file = pd.read_csv(path_to_file)
     print("File opened successfully.")
except Exception as e:
     print("Error occurred while opening the file:", e)

unic_primary = the_file['Primary Type'].unique()
unic_location = the_file['Location Description'].unique()
unic_description = the_file['Description'].unique()

print("XXXXXXXXXXXPrimary Types:XXXXXXXXXXXXXXXXXXXXXX")
print(unic_primary)

print("XXXXXXXXXXXXXXXXXXLocation Descriptions:XXXXXXXXXXXXXXXXXXXXXX")
print(unic_location)

print("XXXXXXXXXXXXXXXXXDescriptions:XXXXXXXXXXXXXX")
print(unic_description)


mapping_for_primary={
     #gun related
     'BATTERY' : 'wepon related',
    'THEFT' : 'wepon related',
    'ASSAULT' : 'wepon related',
     'BURGLARY' : 'wepon related',
    'ROBBERY' : 'wepon related',
    'WEAPONS VIOLATION' : 'wepon related',
     'ARSON' : 'wepon related',
    'RITUALISM' :'wepon related',

     #drug related
     'NARCOTICS' : 'NARCO',
    'OTHER NARCOTIC VIOLATION' : 'NARCO',
     #sexualy related
     'SEX OFFENSE' : 'SEXUAL CRIM ',
    'PROSTITUTION' : 'SEXUAL CRIM',
     'CRIM SEXUAL ASSAULT' : 'SEXUAL CRIM',
    'CRIMINAL SEXUAL ASSAULT' : 'SEXUAL CRIM',
    'HUMAN TRAFFICKING' : 'SEXUAL CRIM',
     'DOMESTIC VIOLENCE':'SEXUAL CRIM',
     # light crimes
     'DECEPTIVE PRACTICE' : 'LIGHT CRIM',
    'OTHER OFFENSE' : 'LIGHT CRIM',
    'PUBLIC PEACE VIOLATION' : 'LIGHT CRIM',
     'GAMBLING' : 'LIGHT CRIM',
    'LIQUOR LAW VIOLATION' : 'LIGHT CRIM',
    'NON - CRIMINAL' : 'LIGHT CRIM',
     'STALKING' : 'LIGHT CRIM',
    'NON-CRIMINAL' : 'LIGHT CRIM',
    'NON-CRIMINAL (SUBJECT SPECIFIED)' : 'LIGHT CRIM',
     'NON-CRIMINAL (SUBJECT SPECIFIED)' : 'LIGHT CRIM',

     # vilence/seriuce crime
     'CRIMINAL DAMAGE' : 'vilence/seriuce crime',
    'CRIMINAL TRESPASS' : 'vilence/seriuce crime',
    'INTERFERENCE WITH PUBLIC OFFICER' : 'vilence/seriuce crime',
     'OFFENSE INVOLVING CHILDREN' : 'vilence/seriuce crime',
    'KIDNAPPING' : 'vilence/seriuce crime',
    'INTIMIDATION' : 'vilence/seriuce crime',
     'HOMICIDE' : 'vilence/seriuce crime',
    'MOTOR VEHICLE THEFT' :'vilence/seriuce crime',
}

the_file['Primary Type']=the_file['Primary Type'].map(mapping_for_primary)



def replace_keywords(description):
    keyword_map = {
        'RESIDENCE': 'PRIVET PROP',
        'BUS': 'PUBLIC',
        'SIDEWALK': 'PUBLIC',
        'APARTMENT' : 'PRIVET PROP',
        'STORE': 'PUBLIC',
        'STREET': 'PUBLIC',
        'GARAGE' : 'PUBLIC',
        'RESTAURANT' : 'PUBLIC',
        'ALLEY' : 'PUBLIC',
        'PARK' : 'PUBLIC',
        'CLUB' : 'PUBLIC',
        'OFFICE' : 'WORK',
        'HOTEL' : 'VACATION',
        'BAR' : 'PUBLIC',
        'TAVERN' : 'PUBLIC',
        'CHURCH' : 'PUBLIC',
        'ARENA' : 'PUBLIC',
        'HOSPITAL' : 'PUBLIC',
        'STATION' : 'PUBLIC',
        'TRAIN' : 'PUBLIC',
        'SHOP' : 'PUBLIC',
        'BANK' : 'PUBLIC',
        'SCHOOL' : 'PUBLIC',
        'PUBLIC' : 'PUBLIC',
        'BUILDING' : 'PRIVET PROP',
        'AIRPORT' : 'PUBLIC',
        'POLICE' : 'GOV',
        'GARAGE' : 'PUBLIC',
        'MEDICAL':'GOV',
        'FARM':'WORK',
        'COLLEGE':'PUBLIC',
        'SCHOOL':'PUBLIC',
        'WATER':'PUBLIC',
        'MOTEL':'VACATION',
        'CLUB':'PUBLIC',
        'BASEMENT':'PRIVET PROP',
        'FACTORY':'WORK',
        'TRAILER':'PRIVET PROP',
        'TAXI CAB':'PUBLIC',
        'HOUSE':'PRIVET PROP',
        'DOCK':'WORK',
        'HALL':'PUBLIC',
        'RIVER':'VACATION',
        'SEWER':'PUBLIC',
        'GOVERNMENT':'GOV',
        'DUMPSTER':'PUBLIC',
        'HOME':'PRIVET PROP',
        'LAKE':'VACATION',
        'POLICE':'GOV',




        # Add more keyword mappings as needed
    }
    for keyword, replacement in keyword_map.items() :
        pattern = re.compile(r'\b%s\b' % re.escape(keyword), re.IGNORECASE)
        if pattern.search(description) :
            description = replacement

    return description

the_file['Location Description']=the_file['Location Description'].apply(replace_keywords)



the_file.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big_saparated_cleaned_latest_version.csv',index=False)

In [ ]:
##### some more cleaning data and prepering for machin learning ############################################

path_to_file = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big_saparated_cleaned_latest_version.csv'

try:
     the_file = pd.read_csv(path_to_file)
     print("File opened successfully.")
except Exception as e:
     print("Error occurred while opening the file:", e)


how_much_unic_we_got=the_file['Location Description'].unique()
print(how_much_unic_we_got)

def replace_keywords2(description):
    keyword_map = {
        'VEHICLE':'PUBLIC',
        'TAXICAB':'PUBLIC',
        'RESIDENTIAL YARD':'PRIVET PROP',
        'HALLWAY/STAIRWELL/ELEVATOR':'PRIVET PROP',
        'PARKING LOT':'PUBLIC',
        'LIBRARY':'PUBLIC',
        'RESIDENTIAL':'PRIVET PROP',
        'CONSTRUCTION':'WORK',
        'WAREHOUSE':'WORK'




        # Add more keyword mappings as needed
    }
    for keyword, replacement in keyword_map.items() :
        pattern = re.compile(r'\b%s\b' % re.escape(keyword), re.IGNORECASE)
        if pattern.search(description) :
            description = replacement

    return description

the_file['Location Description']=the_file['Location Description'].apply(replace_keywords2)
how_much_unic_we_got=the_file['Location Description'].unique()
print('efter_funcccc')
print(how_much_unic_we_got)

the_file.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/test_items_efter.csv',index=False)


how_much_unic_we_got=the_file['Description'].unique()
print('description from  here xxxxxxxxxx')
print(how_much_unic_we_got)

def replace_keywords3(description):
    keyword_map = {
        'DOMESTIC':'HIGH',
        'POCKET-PICKING':'LIGHT',
        'HEROIN':'HIGH',
        'ENTRY':'LIGHT',
        'THEFT':'LIGHT',
        'STRONGARM':'LIGHT',
        'THREAT':'LIGHT',
        'WEAPON':'HIGH',
        'VEHICLE':'LIGHT',
        'PROPERTY':'LIGHT',
        'ARMED':'MEDIUM',
        'HARASSMENT':'LIGHT',
        'OFFENSE':'LIGHT',
        'VIOLATION':'LIGHT',
        'CANNABIS':'MEDIUM',
        'GUN':'MEDIUM',
        'HEROIN':'HIGH',
        'FIREARM':'MEDIUM',
        'CRACK':'MEDIUM',
        'CUTTING':'HIGH',
        'ATTEMPT':'LIGHT',
        'NARCOTICS':'MEDIUM',
        'RESIST':'LIGHT',
        'SEX':'HIGH',
        'ABUSE':'HIGH',
        'VIOLATION':'LIGHT',
        'COCAINE':'HIGH',
        'FRAUD':'LIGHT',
        'CRIMINAL':'MEDIUM',
        'RESIDENCE':'LIGHT',
        'SYNTHETIC DRUGS':'MEDIUM',
        'SOLICIT':'LIGHT',
        'ENDANGERING':'LIGHT',
        'SEXUAL':'HIGH',
        'ABANDONMENT':'MEDIUM',
        'PREDATORY':'HIGH',
        'SIMPLE':'LIGHT',
        'HANDGUN':'MEDIUM',
        # Add more keyword mappings as needed
    }
    for keyword, replacement in keyword_map.items() :
        pattern = re.compile(r'\b%s\b' % re.escape(keyword), re.IGNORECASE)
        if pattern.search(description) :
            description = replacement
    return description

try:
   the_file['Description'] = the_file['Description'].apply(replace_keywords3)
   unique_descriptions = the_file['Description'].unique()
   print('Keywords replaced successfully.')
except execption as e:
    print("Error occurred while opening the file:", e)

print('new from hereeeeee description from  here xxxxxxxxxx')
how_much_unic_we_got=the_file['Description'].unique()
print(how_much_unic_we_got)

conditions_for_description = ~the_file['Description'].isin(['LIGHT', 'MEDIUM', 'HIGH'])
count = the_file[conditions_for_description]['Description'].value_counts()
print('desctiption count')
print(count)


conditions_for_Loc_description = ~the_file['Location Description'].isin(['PRIVET PROP', 'PUBLIC', 'GOV','WORK','VACATION'])
count = the_file[conditions_for_Loc_description]['Location Description'].value_counts()
print('location count')
print(count)

conditions_for_Primary = ~the_file['Primary Type'].isin(['wepon related', 'NARCO', 'SEXUAL CRIM','LIGHT CRIM','vilence/seriuce crime'])
count = the_file[conditions_for_Primary]['Primary Type'].value_counts()
print('primary')
print(count)



row_count=the_file.shape[0]
print("number of rows befor drop",row_count)
print(the_file['Primary Type'])
print(the_file['Location Description'])
print(the_file['Description'])


the_file.drop(the_file.loc[conditions_for_Primary].index,inplace=True)
the_file.drop(the_file.loc[conditions_for_Loc_description].index,inplace=True)
the_file.drop(the_file.loc[conditions_for_description].index,inplace=True)
print('XXXXXXXXXXXefter dropXXXXXXX')
print(the_file['Primary Type'])
print(the_file['Location Description'])
print(the_file['Description'])
the_file.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/test_items_try_EDA.csv',index=False)
row_count=the_file.shape[0]
print("number of rows befor drop",row_count)


the_file['Primary Type'] = the_file['Primary Type'].astype('category')
the_file['Primary Type'] = the_file['Primary Type'].cat.codes + 1

the_file['Location Description'] = the_file['Location Description'].astype('category')
the_file['Location Description'] = the_file['Location Description'].cat.codes + 1

the_file['Description'] = the_file['Description'].astype('category')
the_file['Description'] = the_file['Description'].cat.codes + 1


print('XXXXXXXXXXXefter numeric represss XXXXXXX')
print(the_file['Primary Type'])
print(the_file['Location Description'])
print(the_file['Description'])

the_file.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big_try_this_for_learning_machin.csv',index=False)

In [ ]:
##### translating long and lat into location name #######################

def get_address(address):
    neighborhoods = [
        "Albany Park", "Archer Heights", "Armour Square", "Ashburn", "Auburn Gresham", "Austin", "Avalon Park",
        "Avondale", "Belmont Cragin", "Beverly", "Bridgeport", "Brighton Park", "Burnside", "Calumet Heights",
        "Chatham", "Chicago Lawn", "Clearing", "Douglas", "Dunning", "East Garfield Park", "East Side", "Edgewater",
        "Edison Park", "Englewood", "Forest Glen", "Fuller Park", "Gage Park", "Garfield Ridge", "Grand Boulevard",
        "Greater Grand Crossing", "Hegewisch", "Hermosa", "Humboldt Park", "Hyde Park", "Irving Park", "Jefferson Park",
        "Kenwood", "Lake View", "Lincoln Park", "Lincoln Square", "Logan Square", "Loop", "Lower West Side",
        "McKinley Park", "Montclare", "Morgan Park", "Mount Greenwood", "Near North Side", "Near South Side", "Near West Side",
        "New City", "North Center", "North Lawndale", "North Park", "Norwood Park", "O'Hare", "Oakland", "Portage Park",
        "Pullman", "Riverdale", "Rogers Park", "Roseland", "South Chicago", "South Deering", "South Lawndale",
        "South Shore", "Streeterville", "Uptown", "Washington Heights", "Washington Park", "West Elsdon",
        "West Englewood","West Garfield Park", "West Lawn", "West Pullman", "West Ridge", "West Town", "Woodlawn"
    ]
    address = address.replace("(", "")
    address=address.replace(")","")
    #print(address, 'togther')

    location=geocoder.osm(address,method='reverse')
    location=str(location)
    #print(location,"copy from here ")
    location=location.split(",")
    need=None
    if location is not None:
        for i in range(len(location)):
            print(location[i].strip(),'the i is here')
            if location[i]==" Chicago":
                need=location[i-1]
                #print(need,"waht is need")
                break
    if need is not None :
        print(need, "what we found")
    else :
        print("no matches found")
    return need







file_path = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/exel_4000kdata_pls_work.csv'
df = pd.read_csv(file_path)
for index, row in df.iterrows():
    address = row['Location']
    df.at[index, 'NEW LOCATION'] = get_address(address)
#df['Location']=df['Location'].apply(translate_add)

df.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/exel_4000kdata_translated.csv',index=False)

In [ ]:
### orgenizing the places to 5 major districts ###################
#####then predictin other places only by long and lat cos it would take a lot of time to do it other way ####
path_to_file = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/exel_4000kdata_translated.csv'

try:
     the_file = pd.read_csv(path_to_file)
     print("File opened successfully.")
except Exception as e:
     print("Error occurred while opening the file:", e)

the_file = the_file.drop(the_file[the_file['Location'] == 'UNknown'].index)

the_file[['Latitude', 'Longitude']] = the_file['Location'].str.split(',', expand=True)
the_file['Latitude'] = the_file['Latitude'].astype(str)
the_file['Longitude'] = the_file['Longitude'].astype(str)
the_file['Latitude'] = the_file['Latitude'].str.strip()
the_file['Longitude'] = the_file['Longitude'].str.strip()
the_file['Latitude'] = the_file['Latitude'].str.replace('(', '')
the_file['Longitude'] = the_file['Longitude'].str.replace(')', '')
the_file['Latitude'] = pd.to_numeric(the_file['Latitude'])
the_file['Longitude'] = pd.to_numeric(the_file['Longitude'])
uniq_locations=the_file['NEW LOCATION'].unique()
print(uniq_locations)
mapping_for_location={
    " Loop":"Upper Town",
    " Near":"Upper Town",
    " North":"Upper Town",
    " Side":"Upper Town",
    " River":"Upper Town",
    " North":"Upper Town",
    " Streeterville":"Upper Town",
    " Gold":"Upper Town",
    " Coast":"Upper Town",
    " Old":"Upper Town",
    " Town":"Upper Town",
    " Lincoln":"Upper Town",
    " Park":"Upper Town",
    " Lake":"Upper Town",
    " View":"Upper Town",
    " Uptown":"Upper Town",
    " North":"Upper Town",
    " Center":"Upper Town",
    " Lincoln":"Upper Town",
    " Square":"Upper Town",
    " Edgewater":"Upper Town",
    " Rogers":"Upper Town",
    " Park":"Upper Town",
    " West":"Upper Town",
    " Ridge":"Upper Town",
    " North":"Upper Town",
    " Park":"Upper Town",
    " Albany":"Upper Town",
    " Park":"Upper Town",
    " Jefferson":"Upper Town",
    " Park":"Upper Town",
    " Norwood":"Upper Town",
    " Park":"Upper Town",
    " Edison":"Upper Town",
    " Park":"Upper Town",
    " South Shore":"Lower Town",
    " Hyde Park":"Lower Town",
    " Woodlawn":"Lower Town",
    " Kenwood":"Lower Town",
    " Bronzeville":"Lower Town",
    " Washington Park":"Lower Town",
    " Fuller Park":"Lower Town",
    " Grand Boulevard":"Lower Town",
    " Englewood":"Lower Town",
    " West Englewood":"Lower Town",
    " Gage Park":"Lower Town",
    " Chicago Lawn":"Lower Town",
    " West Lawn":"Lower Town",
    " Ashburn":"Lower Town",
    " Marquette Park":"Lower Town",
    " Garfield Ridge":"Lower Town",
    " Clearing":"Lower Town",
    " West Elsdon":"Lower Town",
    " Archer Heights":"Lower Town",
    " Brighton Park":"Lower Town",
    " Austin":"West Side",
    " Humboldt Park":"West Side",
    " Garfield Park":"West Side",
    " West Garfield Park":"West Side",
    " North Lawndale":"West Side",
    " East Garfield Park":"West Side",
    " Belmont Cragin":"West Side",
    " Hermosa":"West Side",
    " Logan Square":"West Side",
    " South Deering":"East Side",
    " East Side":"East Side",
    " South Chicago":"East Side",
    " Hegewisch":"East Side",
    " Riverdale":"East Side",
    " Pullman":"East Side",
    " Burnside":"East Side",
    " Calumet Heights":"East Side",
    " Roseland":"East Side",
    " Greater Grand Crossing":"Middle City",
    " Chatham":"Middle City",
    " Avalon Park":"Middle City",
    " South Shore Historical Bungalow District":"Middle City",
    " South Commons":"Middle City",
    " Bridgeport":"Middle City",
    " McKinley Park":"Middle City",
    " Washington Heights":"Middle City",
    " Morgan Park":"Middle City",
    " Beverly":"Middle City",
    " Mount Greenwood":"Middle City",
    " Washington Park":"Middle City",
    " Oakland":"Middle City",
    " Douglas":"Middle City",
    " Kenwood":"Middle City",
    " Grand Boulevard":"Middle City",
    " Fuller Park":"Middle City",
    " Woodlawn":"Middle City",
    " Bronzeville":"Middle City",
    " Chicago Lawn":"Middle City",
    " Avondale": "Upper Town",
    " Auburn Gresham": "Lower Town",
    " South Lawndale": "Lower Town",
    " Lower Town": "Lower Town",
    " West Side": "West Side",
    " Middle City": "Middle City",
    " Jefferson Park": "Upper Town",
    " East Side": "East Side",
    " West Town": "West Town",
    " Portage Park": "Upper Town",
    " Forest Glen": "Upper Town",
    " Midway Village": "Middle City",
    " Albany Park": "Upper Town",
    " West Ridge": "Upper Town",
    " Near North Side": "Upper Town",
    " Lake View": "Upper Town",
    " Near West Side": "Upper Town",
    " West Pullman": "Lower Town",
    " Near South Side": "Upper Town",
    " North Park": "Upper Town",
    " River North": "Upper Town",
    " Lincoln Park": "Upper Town",
    " North Center": "Upper Town",
    " Rogers Park": "Upper Town",
    " Norwood Park": "Upper Town",
    " Bryn Mawr East": "Upper Town",
    " Altgeld Gardens": "Lower Town",
    " Lower West Side": "Lower Town",
    " Parkside": "Upper Town",
    " London Towne Houses": "Upper Town",
    " Beat 2523": "Upper Town",
    " Andersonville": "Upper Town",
    " Kinzie Industrial Corridor": "Upper Town",
    " Armour Square": "Upper Town",
    " Irving Park": "Upper Town",
    " Beat 2512": "Upper Town",
    " Lincoln Square": "Upper Town",
    " Dunning": "Upper Town",
    " Bryn Mawr West": "Upper Town",
    " Michigan Ave": "Upper Town",
    " Six Corners": "Upper Town",
    " Montclare": "Upper Town",
    " O'Keeffe": "Upper Town",
    " The Yards Plaza": "Upper Town",
    " O'Hare": "Upper Town",
    " Beat 2521": "Upper Town",
    " Beat 2522": "Upper Town",
    " The Island": "Upper Town",
    " Beat 2515": "Upper Town",
    " The Grant": "Upper Town",
    " Catherine Courts Apartments": "Upper Town",
    " Edison Park": "Upper Town",
    " Beat 2532": "Upper Town",
    " Stockyards Industrial Corridor": "Upper Town",
    " Beat 2514": "Upper Town",
    " Jeffrey - Cyril Historic District": "Upper Town",
    " Beat 2511": "Upper Town",
    " Concordia Place Apartments": "Upper Town",
    " Chinatown": "Upper Town",
    " Dearborn Homes": "Upper Town",
    " Jackson Park Highlands District": "Upper Town",
    " Madison Terrace Apartments": "Upper Town",
    " Martin Luther King Jr Plaza Apartments": "Upper Town",
    " Gateway Centre Plaza": "Upper Town",
    " Lake Meadows": "Upper Town",
    " Beat 2535": "Upper Town",
    " Essex": "Upper Town",
    " Chatham Park Village Cooperative": "Upper Town",
    " Peoples Gas": "Upper Town",
    " CTA West Shops": "Upper Town",
    " New City":"Middle City",

}
the_file['simple location']=the_file['NEW LOCATION'].replace(mapping_for_location)
print("new unique in simple location ",the_file['simple location'].unique())
keep=['Middle City', 'West Side', 'Upper Town', 'Lower Town', 'East Side']
mask = ~the_file['simple location'].isin(keep)
the_file.drop(the_file[mask].index, inplace=True)
print("new unique in simple location ",the_file['simple location'].unique())

the_file.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/exel_4000kdata_translated_and_seperated.csv', index=False)

x = the_file[['Latitude', 'Longitude']]
y = the_file['simple location']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

decision_tree = DecisionTreeClassifier()

param_grid = {
     'criterion': ['gini', 'entropy'],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_depth': [None, 5, 10, 15, 20]
 }

grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_decision_tree = grid_search.best_estimator_
accuracy = best_decision_tree.score(x_test, y_test)
print("Test Set Accuracy: ", accuracy)
path_to_file = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/importent copy dont touch/full_data_upgraded_cleaned.csv'

try:
     big_file = pd.read_csv(path_to_file)
     print("File opened successfully.")
except Exception as e:
     print("Error occurred while opening the file:", e)

mini_big_file=big_file.head(65000)
try:
    mini_big_file.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big.csv', index=False)
    print("we did save it so it hasse to bi some where there ")

except:
    print("Failed to save file")
path_to_file = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big.csv'

try:
     big_file = pd.read_csv(path_to_file)
     print("XXXXXXXXXXXFile opened successfully.XXXXX")
except Exception as e:
     print("Error occurred while opening the file:", e)
try:

    mini_big_file[['Latitude', 'Longitude']] = mini_big_file['Location'].str.split(',', expand=True)
    mini_big_file['Latitude'] = mini_big_file['Latitude'].astype(str)
    mini_big_file['Longitude'] = mini_big_file['Longitude'].astype(str)
    mini_big_file['Latitude'] = mini_big_file['Latitude'].str.strip()
    mini_big_file['Longitude'] = mini_big_file['Longitude'].str.strip()
    mini_big_file['Latitude'] = mini_big_file['Latitude'].str.replace('(', '')
    mini_big_file['Longitude'] = mini_big_file['Longitude'].str.replace(')', '')
    mini_big_file['Latitude'] = pd.to_numeric(mini_big_file['Latitude'])
    mini_big_file['Longitude'] = pd.to_numeric(mini_big_file['Longitude'])


    print("we did all the bullshit it should be there")
except:
    print("we failed some where in the bullshit ")
try:
        mini_big_file = mini_big_file.drop(mini_big_file[mini_big_file['Location'] == 'UNknown'].index)
        print("it worked the unknown should be deleted")
except:
        print("removing failed for unknown")

x_predict = mini_big_file[['Latitude', 'Longitude']]
mini_big_file['simple location'] = best_decision_tree.predict(x_predict)

try:

    mini_big_file.to_csv('D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big_saparated.csv', index=False)
    print("we saved the file at the end ")
except:
    print("we failed to save this end file ")

In [ ]:
#### first try machin learning ##########LogisticRegression###############decision tree############
file_path = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/test_items_try_muchinlearing.csv'
df = pd.read_csv(file_path)

print(df['Primary Type'].dtype)
print(df['Location Description'].dtype)
print(df['Description'].dtype)
print(df['Hour'].dtype)

x = df[['Primary Type', 'Location Description', 'Description']]
y = df['Hour']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
acc_score = accuracy_score(y_test, y_pred)
print("Decision Tree Accuracy Score:", acc_score)

x = df[['Primary Type', 'Location Description', 'Description']]
y = (df['Hour'] >= 12).astype(int)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
acc_score = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy Score:", acc_score)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

regression = LinearRegression()
regression.fit(X_train, y_train)
y_pred = regression.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Linear Regression Mean Squared Error:", mse)
print("Linear Regression R-squared Score:", r2)

plt.scatter(y_test, y_pred)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Linear Regression: Actual vs Predicted')
plt.show()

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("KNN Accuracy Score:", accuracy)

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('KNN: Confusion Matrix')
plt.show()

best_score = 0
best_depth = 0
best_min_samples_split = 0

for depth in range(1, 11):
    for min_samples_split in range(2, 11):
        tree = DecisionTreeClassifier(max_depth=depth, min_samples_split=min_samples_split)
        tree.fit(X_train, y_train)
        y_pred = tree.predict(X_test)
        score = accuracy_score(y_test, y_pred)

        if score > best_score:
            best_score = score
            best_depth = depth
            best_min_samples_split = min_samples_split

print("Best Accuracy Score:", best_score)
print("Best Tree Depth:", best_depth)
print("Best Minimum Samples Split:", best_min_samples_split)

# Train and evaluate the decision tree with the best parameters
tree_best = DecisionTreeClassifier(max_depth=best_depth, min_samples_split=best_min_samples_split)
tree_best.fit(X_train, y_train)
y_pred_best = tree_best.predict(X_test)

acc_score_best = accuracy_score(y_test, y_pred_best)
print("Decision Tree Accuracy Score:", acc_score_best)

In [ ]:
####### second try machin learning #########

path_to_file = 'D:/programs/CHROM EXECUTEBLES/files for project/fullstart/4k data/mini_big_try_this_for_learning_machin.csv'

try:
    big_file = pd.read_csv(path_to_file)
    print("File opened successfully.")
except Exception as e:
    print("Error occurred while opening the file:", e)

x = big_file[['Primary Type', 'Description', 'Location Description', 'Hour', 'Latitude', 'Longitude', 'Domestic', 'Arrest']]
y = big_file['simple location']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

decision_tree = DecisionTreeClassifier()

param_grid = {
    'criterion': ['gini', 'entropy'],
    'min_samples_leaf': [1, 2, 3],
    'max_depth': [1, 2, 3, 4, 5,]
}

grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, scoring='accuracy')
grid_search.fit(x_train, y_train)

best_decision_tree = grid_search.best_estimator_
accuracy = best_decision_tree.score(x_test, y_test)
print("Decision Tree Test Set Accuracy: ", accuracy)

logistic_regression = LogisticRegression(multi_class='multinomial', solver='lbfgs',max_iter=1000)
logistic_regression.fit(x_train, y_train)
predictions = logistic_regression.predict(x_test)
accuracy = logistic_regression.score(x_test, y_test)
print("Logistic Regression Test Set Accuracy: ", accuracy)

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# Create an array to indicate correct predictions
correct_predictions = predictions == y_test

# Calculate accuracy for each label
label_accuracy = {}
unique_labels = np.unique(y_test)
for label in unique_labels:
    label_accuracy[label] = np.mean(correct_predictions[y_test == label])

# Plot bar graph
plt.figure(figsize=(10, 6))
plt.bar(label_accuracy.keys(), label_accuracy.values())
plt.xlabel("Labels")
plt.ylabel("Accuracy")
plt.title("Prediction Accuracy for Each Label")
plt.show()